# Youtube Data Cleaning

This file was used to clean the Youtube data obtained from Google Takeout. The cleaned data and results are saved as a csv file to be used in sleep_inference.ipynb.

In [ ]:
# Import necessary libraries
import pandas as pd
import json
import numpy as np
from datetime import datetime, timedelta

## Clean the watch history data

In [ ]:
# REPLACE WITH YOUR FILE PATH
filename = "../EZ_data/watch-history-json.json"

with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# List to store extracted data
extracted_data = []

# Loop through each entry in the JSON data and extract the relevant fields
for entry in data:
    title = entry.get("title", None)
    url = entry.get("titleUrl", None)
    time_str = entry.get("time", None)

    # Try to parse the date (you can adjust the format if needed)
    date_time = None
    if time_str:
        try:
            date_time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            date_time = time_str  # Keep original string if parsing fails

    # Only append if there is a valid title (to avoid extra records)
    if title:
        extracted_data.append({"title": title, "url": url, "date_time": date_time})

# Convert the extracted data to a DataFrame
df_watch = pd.DataFrame(extracted_data)
df_watch.head()


In [ ]:
# Convert date_time to datetime
df_watch['date_time'] = pd.to_datetime(df_watch['date_time'], errors='coerce', utc=True)

# Convert from UTC to US/Eastern timezone
df_watch['date_time'] = df_watch['date_time'].dt.tz_convert('US/Eastern')

# Extract the date and time separately
df_watch['date'] = df_watch['date_time'].dt.date
df_watch['time'] = df_watch['date_time'].dt.time
df_watch.head(2)

In [ ]:
# Extract title
pattern = r'^(Watched)\s+(.*)$'

df_watch[["type", "title"]] = df_watch["title"].str.extract(pattern)
df_watch.head(2)

In [ ]:
# Add videos per day
videos_per_day = df_watch.groupby('date').size().reset_index(name='total_videos_watched')
videos_per_day['date'] = pd.to_datetime(videos_per_day['date'], errors='coerce')
start_date = "2025-01-01"
end_date = "2025-02-17"
start_date = pd.to_datetime(start_date).date()
start_date = pd.Timestamp(start_date)
end_date = pd.to_datetime(end_date).date()
end_date = pd.Timestamp(end_date)
videos_per_day = videos_per_day[(videos_per_day["date"] >= start_date) & (videos_per_day["date"] <= end_date)]

videos_per_day.head()

## Clean the search history data

In [ ]:
# REPLACE WITH YOUR FILE PATH
filename = "../EZ_data/search-history.json"

with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# List to store extracted data
extracted_data = []

# Loop through each entry in the JSON data and extract the relevant fields
for entry in data:
    title = entry.get("title", None)
    url = entry.get("titleUrl", None)
    time_str = entry.get("time", None)

    # Try to parse the date (you can adjust the format if needed)
    date_time = None
    if time_str:
        try:
            date_time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            date_time = time_str  # Keep original string if parsing fails

    # Only append if there is a valid title (to avoid extra records)
    if title:
        extracted_data.append({"title": title, "url": url, "date_time": date_time})

# Convert the extracted data to a DataFrame
df_search = pd.DataFrame(extracted_data)
df_search.head()

In [ ]:
# Separate the search history and watch history using regex
pattern = r'^(Searched|Watched)\s+(.*)$'

df_search[["type", "title"]] = df_search["title"].str.extract(pattern)
df_search.head()

In [ ]:
# Convert date_time to datetime
df_search['date_time'] = pd.to_datetime(df_search['date_time'], errors='coerce', utc=True)

# Convert from UTC to US/Eastern timezone
df_search['date_time'] = df_search['date_time'].dt.tz_convert('US/Eastern')

# Extract the date and time separately
df_search['date'] = df_search['date_time'].dt.date
df_search['time'] = df_search['date_time'].dt.time
df_search.head(2)

## Combine the search and watch history data exported from Google Takeout

In [ ]:
# Combine the dataframes
df = pd.concat([df_search, df_watch], axis=0)

# Remove duplicate records
df = df.drop_duplicates()

# Sort by date and time and filter out time range
start_date = "2025-01-01"
end_date = "2025-02-17"
start_date = pd.to_datetime(start_date).date()
end_date = pd.to_datetime(end_date).date()
df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
df = df.sort_values("date_time")

# Extract the day of the week
df['day_of_week'] = df['date_time'].dt.day_name()

df.head()

## Create summary for analyzing Youtube summary statistics

In [ ]:
# Define a threshold for a new session; events more than 30 minutes apart indicate a new session
session_gap = timedelta(minutes=30)

# Compute the difference in time between consecutive events
df['time_diff'] = df['date_time'].diff()

# Create a boolean column indicating if an event starts a new session
df['new_session'] = (df['time_diff'] > session_gap) | (df['time_diff'].isna())

# Create a session ID by cumulatively summing the new_session markers
df['session_id'] = df['new_session'].cumsum()

# Calculate session start, end, and duration by grouping on session_id
session_stats = df.groupby('session_id')['date_time'].agg(['min', 'max'])
session_stats['session_duration'] = session_stats['max'] - session_stats['min']
session_stats['date'] = session_stats['max'].dt.date

session_stats.head()

In [ ]:
# Extract the date from the session's end time (max)
session_stats['date'] = session_stats['max'].dt.date

# For each date, identify the session with the latest end time ('max')
last_sessions = session_stats.loc[session_stats.groupby('date')['max'].idxmax()]

# Get the session IDs corresponding to the last session of each day
last_session_ids = last_sessions.index

# Filter the original DataFrame to include only the records from these last sessions
df_last_sessions = df[df['session_id'].isin(last_session_ids)]

# Now, for each of these last sessions, pick only the final record (i.e. the one with the max timestamp)
last_record_per_session = df_last_sessions.groupby('session_id', as_index=False).last()

last_session_df = last_record_per_session[['date_time', 'session_id', 'time_diff', 'date']]
last_session_df.head()

## Create summary for overall Youtube activity

In [ ]:
# Function for getting the first and last visits on Youtube 
def get_first_and_last_visits(df):
    # find first and last visited websites per date
    first_visits = df.loc[df.groupby("date")["time"].idxmin()]
    last_visits = df.loc[df.groupby("date")["time"].idxmax()]

    # select relevant columns
    first_visits = first_visits[["date", "time", "url", "title", "type"]].rename(columns={
        "time": "time_first",
        "url": "url_first",
        "title": "title_first",
        "type": "type_first"
    })
    
    last_visits = last_visits[["date", "time", "url", "title", "type"]].rename(columns={
        "time": "time_last",
        "url": "url_last",
        "title": "title_last",
        "type": "type_last"
    })

    # merge both DataFrames on visit_date to get one row per date
    youtube_summary = pd.merge(first_visits, last_visits, on="date")
    return youtube_summary

youtube_summary_df = get_first_and_last_visits(df)

youtube_summary_df.head()

In [ ]:
# Add the data for the last session each night
youtube_summary_df = pd.merge(youtube_summary_df, last_session_df, on="date", how="inner")
youtube_summary_df["had_session"] = np.where(youtube_summary_df["session_id"].isna(), False, True)
youtube_summary_df = youtube_summary_df.drop_duplicates(subset=['date'])

# Rename column for time_diff
youtube_summary_df = youtube_summary_df.rename(columns={"time_diff": "session_duration"})

youtube_summary_df.head()

## Combine Youtube summary with sleep data

In [ ]:
# Import sleep data

# CHANGE THIS TO YOUR FILE PATH
sleep_filename = "../elaine_sleep_data.csv"

sleep_df = pd.read_csv(sleep_filename)
sleep_df["Date"] = pd.to_datetime(sleep_df["Date"])
sleep_df = sleep_df.rename(columns={"Date": "date"})
sleep_df = sleep_df.drop(columns=["Unnamed: 0"])
sleep_df.tail()

In [ ]:
youtube_summary_df["date"] = pd.to_datetime(youtube_summary_df["date"])
df = pd.merge(sleep_df, youtube_summary_df, on="date", how="right")
df.head()

In [ ]:
# Add videos per day
df = pd.merge(df, videos_per_day,  on="date", how="right")
df = df[df['total_videos_watched'] <= 180]
df.head()

In [ ]:
# RENAME FILE AS APPROPRIATE
df.to_csv("../youtube_sleep_EZ.csv")